In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import spacy

import nltk, re
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /Users/coby.lu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [323]:
nlp = spacy.load("en_core_web_md")

In [77]:
df = pd.read_csv("Engineering Secret Snowflake 2019 (Responses) - Form Responses 1.csv")

In [78]:
df = df.loc[df["Do you want to participate in Engineering's Secret Snowflake?"] == "Yes", ["Email Address", "List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]].reset_index(drop=True)

In [527]:
print(df.shape)
df.tail(25)

(42, 3)


,Email Address,List 4 facts about yourself to help your secret snowflake find the perfect gift for you!,proc
17,kevin.reed@appian.com,"I really like to run and keep active, Texas Te...",run active Texas Tech reading learning history...
18,michael.nye@appian.com,"I like electronics, cars/motorcycles, guitar r...",electronics cars motorcycles guitar related th...
19,allison.gisinger@appian.com,I like board games (preferably for 2+ people)....,board games 2 people trying new creative thing...
20,jorge.sanchez@appian.com,"Love to travel, love racquet sports, love beac...",travel racquet sports beach coffee
21,serdar.benderli@appian.com,I enjoy listening to music on vinyl - I enjoy ...,listening music vinyl rock jazz blues metal cl...
22,srikanth.chelluri@appian.com,"My favorite show is ""The Office"" (the US editi...",favorite Office edition Unlike rest world Mich...
23,bill.warshaw@appian.com,"I like hot sauces (really hot), Rocket League,...",hot sauces hot Rocket League history beer
24,kelley.macewen@appian.com,- I enjoy cycling and running - trying to keep...,cycling running trying fit cold winter months ...
25,katie.ward@appian.com,"Hobbies: Baking, crochet; Candy: peanut butter...",Baking crochet Candy peanut butter chocolate C...
26,abigail.butler@appian.com,I am getting ready to travel out of the countr...,getting ready travel country end December dog ...


In [449]:
df_simp = df.loc[[1, 2, 8, 12, 20, 31, 39]].reset_index(drop=True)
df_simp.head(7)

,Email Address,List 4 facts about yourself to help your secret snowflake find the perfect gift for you!,proc
0,nice.yslas@appian.com,"Hobbies: traveling, skydiving, reading. favor...",video game Pokemon Animal Crossing- plushie aw...
1,rachel.bittner@appian.com,"Hobbies: video games - Pokemon, Animal Crossin...",eat travel running hiking cozy thing sock swea...
2,erin.kocis@appian.com,"I love eating, traveling, running, hiking, coz...",eating traveling running hiking cozy things so...
3,franklin.zheng@appian.com,"My main hobbies are Video games, anime, DIY, b...",main Video games anime DIY boardgames Tshirt s...
4,jorge.sanchez@appian.com,"Love to travel, love racquet sports, love beac...",travel racquet sports beach coffee
5,katie.freix@appian.com,"I enjoy reading, board games, running and wood...",reading board games running woodworking favori...
6,coby.lu@appian.com,I like strategy/deception board games \nI like...,strategy deception board games MCU Deadpool su...


In [450]:
corpus = df_simp["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]

In [451]:
corpus.loc[2]

"I love eating, traveling, running, hiking, cozy things like socks and sweaters and fuzzy plushies, and honestly there aren't many things I don't like (besides fish and peppers)! "

In [249]:
spacy_corpus = corpus.apply(nlp)

In [250]:
spacy_similarity = np.zeros((len(spacy_corpus), len(spacy_corpus)))
for (i, doc) in enumerate(spacy_corpus):
    for (j, other_doc) in enumerate(spacy_corpus):
        spacy_similarity[i, j] = doc.similarity(other_doc)

In [251]:
print(spacy_similarity)

[[1.         0.84714053 0.82833776 0.84996164 0.83255159 0.85296411
  0.8681463 ]
 [0.84714053 1.         0.77341081 0.8412753  0.73243103 0.81402733
  0.81873769]
 [0.82833776 0.77341081 1.         0.90178292 0.87744501 0.93312464
  0.89189085]
 [0.84996164 0.8412753  0.90178292 1.         0.84242319 0.93597764
  0.91473291]
 [0.83255159 0.73243103 0.87744501 0.84242319 1.         0.85610431
  0.8147087 ]
 [0.85296411 0.81402733 0.93312464 0.93597764 0.85610431 1.
  0.94035053]
 [0.8681463  0.81873769 0.89189085 0.91473291 0.8147087  0.94035053
  1.        ]]


In [278]:
np.argsort(spacy_similarity[5, :])

array([1, 0, 4, 2, 3, 6, 5])

In [452]:
spacy_corpus = corpus.apply(nlp)

In [453]:
sentence = spacy_corpus[1]

In [454]:
print(sentence)

Hobbies: video games - Pokemon, Animal Crossing- plushies are awesome
Candy/Color: Sour Candy/Blue- sour patch kids/sour punch straws


In [331]:
for word in sentence:
    print(word.lemma_)

hobby
:
travel
,
skydiving
,
read
.
 
favorite
color
/
candy
:
red
/
kitkat
,
i
love
harry
potter
.
caffeine
addict
.


In [351]:
all_lemmas = set()

In [352]:
for sentence in spacy_corpus:
    for word in sentence:
        all_lemmas.add(str(word.lemma_))



enjoy
learn
about
history
of
different
country
.
ancient
remedy
for
common
illness
hobby
:
travel
,
skydiving
read
 
favorite
color
/
candy
red
kitkat
i
love
harry
potter
caffeine
addict
video
game
-
Pokemon
Animal
Crossing-
plushie
be
awesome


Candy
sour
blue-
patch
kid
punch
straw
-PRON-
compete
in
powerlifting
play
electric
guitar
cat
effectively
son
and
Magic
the
gathering
outside
work
like
to
cook
watch
Netflix
Hulu
draw
on
ipad
knit
teal
green
Ferrero
Rocher
get
bath
&
body
product
bomb
gift
engineering
;
any(all
'
em!!!!111
)
desk
trinket
--
sure
!
have
a
passion
tasteful
elegant
knickknack
from
goodwill
Tennis
Cricket
Beaches
...
eat
running
hiking
cozy
thing
sock
sweater
fuzzy
honestly
there
not
many
do
(
besides
fish
pepper
LEGO
bowl
league
over
20
year
Star
Wars
workout
regularly
musician
saxophone
legos
model
building
original
career
choice
architect
Google
Home
>
Alexa
lean
towards
someone
small
smart
device
car
build
pc
electronic
chocolate
main
course
change
mind
effici

In [354]:
list_lemma = list(all_lemmas)

In [355]:
list_lemma.sort()
print(list_lemma)

['\n', '\n\n', ' ', '!', '"', '&', "'", '(', ')', '+', ',', '-', '--', '-PRON-', '.', '..', '...', '/', '1', '10', '2', '20', '3', '300', '3d', '4', '6', ':', ':)', ';', '>', '?', 'Alexa', 'Animal', 'Appian', 'Atul', 'Beaches', 'Blue', 'Board', 'Candy', 'Christmas', 'Cricket', 'Crossing-', 'D&D', 'DC', 'DIY', 'Deadpool', 'December', 'Engineering', 'Ferrero', 'Game', 'Gawande', 'Google', 'Graphic', 'Harry', 'Home', 'Hulu', 'January', 'Khalid', 'LEGO', 'League', 'Lil', 'MCU', 'Magic', 'Malone', 'Marvel', 'Maryland', 'Mechanical', 'Medium', 'Michael', 'Netflix', 'Office', 'Pokemon', 'Post', 'Potter', 'Rocher', 'Rocket', 'Scott', 'Sports', 'Star', 'Starbucks', 'Tech', 'Tee', 'Tennis', 'Texas', 'Travis', 'Ts', 'Turtle', 'Twix', 'US', 'University', 'Wars', 'Yachty', '_', 'a', 'about', 'account', 'active', 'activity', 'actually', 'add', 'addict', 'after', 'alcohol', 'all', 'almost', 'already', 'also', 'always', 'an', 'ancient', 'and', 'anime', 'any', 'any(all', 'anything', 'apart', 'appreciat

In [488]:
stop_lemmas = set(['\n', '\n\n', ' ', '!', '"', '&', "'", '(', ')', '+', ',', '-', '--', '-PRON-', '.', '..', '...', '/', ':', ':)', 
';', '>', '?','_', 'a', 'an', 'also', 'candy', 'color', 'enjoy', 'favorite', 'hobby', 'honestly', 'i','interested', 'interesting', 'into', 'like', 
'love', 'lover', 'need', 'preferably','obsessed','really'])

In [528]:
df_simp = df.loc[[29, 2, 8, 12, 20, 31, 39]].reset_index(drop=True)
df_simp.head(7)

,Email Address,List 4 facts about yourself to help your secret snowflake find the perfect gift for you!,proc
0,sydney.pujadas@appian.com,"foodie, art lover, traveler, music lover",foodie art traveler music
1,rachel.bittner@appian.com,"Hobbies: video games - Pokemon, Animal Crossin...",eat travel running hiking cozy thing sock swea...
2,erin.kocis@appian.com,"I love eating, traveling, running, hiking, coz...",eating traveling running hiking cozy things so...
3,franklin.zheng@appian.com,"My main hobbies are Video games, anime, DIY, b...",main Video games anime DIY boardgames Tshirt s...
4,jorge.sanchez@appian.com,"Love to travel, love racquet sports, love beac...",travel racquet sports beach coffee
5,katie.freix@appian.com,"I enjoy reading, board games, running and wood...",reading board games running woodworking favori...
6,coby.lu@appian.com,I like strategy/deception board games \nI like...,strategy deception board games MCU Deadpool su...


In [552]:
corpus = df["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]

In [553]:
spacy_corpus = corpus.apply(nlp)

In [557]:
df_preproc = df

In [558]:
for i, sentence in enumerate(spacy_corpus):
    new_sentence = []
    for word in sentence:
        if not word.is_stop and str(word.lemma_) not in stop_lemmas:
            new_sentence.append(str(word.lemma_))
    df_preproc.loc[i, "proc"] = " ".join(new_sentence)

In [559]:
df_preproc.head(7)

,Email Address,List 4 facts about yourself to help your secret snowflake find the perfect gift for you!,proc
0,kunal.chitalia@appian.com,Enjoy learning about history of different coun...,learn history different country learn ancient ...
1,nice.yslas@appian.com,"Hobbies: traveling, skydiving, reading. favor...",travel skydiving read red kitkat harry potter ...
2,rachel.bittner@appian.com,"Hobbies: video games - Pokemon, Animal Crossin...",video game Pokemon Animal Crossing- plushie aw...
3,jake.shankman@appian.com,"I compete in powerlifting, I play electric gui...",compete powerlifting play electric guitar cat ...
4,annika.basch@appian.com,"Outside of work I like to cook, watch Netflix ...",outside work cook watch Netflix Hulu draw ipad...
5,dima.chechetkin@appian.com,Hobby: electric engineering; color/candy: red/...,electric engineering red any(all em!!!!111 des...
6,tom.zanger@appian.com,I have a passion for tasteful and elegant knic...,passion tasteful elegant knickknack goodwill


In [560]:
corpus = df_preproc["proc"]
print(corpus)

0     learn history different country learn ancient ...
1     travel skydiving read red kitkat harry potter ...
2     video game Pokemon Animal Crossing- plushie aw...
3     compete powerlifting play electric guitar cat ...
4     outside work cook watch Netflix Hulu draw ipad...
5     electric engineering red any(all em!!!!111 des...
6          passion tasteful elegant knickknack goodwill
7                                Tennis Cricket Beaches
8     eat travel running hiking cozy thing sock swea...
9     LEGO bowl league 20 year Star Wars workout reg...
10    musician saxophone red legos model building or...
11    car game build pc electronic chocolate main co...
12    main video game anime DIY boardgame tshirt siz...
13    Mechanical Engineering thing apart eatable gif...
14    run University Maryland Sports green coffee mu...
15    bake cooking board game k beauty product gener...
16    winter wear wool sock hat big plus will coffee...
17    run active Texas Tech read learn history s

In [535]:
df_simp.head(10)

,Email Address,List 4 facts about yourself to help your secret snowflake find the perfect gift for you!,proc
0,sydney.pujadas@appian.com,"foodie, art lover, traveler, music lover",foodie art traveler music
1,rachel.bittner@appian.com,"Hobbies: video games - Pokemon, Animal Crossin...",video game Pokemon Animal Crossing- plushie aw...
2,erin.kocis@appian.com,"I love eating, traveling, running, hiking, coz...",eat travel running hiking cozy thing sock swea...
3,franklin.zheng@appian.com,"My main hobbies are Video games, anime, DIY, b...",main video game anime DIY boardgame tshirt siz...
4,jorge.sanchez@appian.com,"Love to travel, love racquet sports, love beac...",travel racquet sport beach coffee
5,katie.freix@appian.com,"I enjoy reading, board games, running and wood...",read board game run woodworke pastel neutral s...
6,coby.lu@appian.com,I like strategy/deception board games \nI like...,strategy deception board game MCU Deadpool sup...


In [536]:
for sent in corpus:
    print(sent)
    print()

foodie art traveler music

video game Pokemon Animal Crossing- plushie awesome Candy sour Candy blue- sour patch kid sour punch straw

eat travel running hiking cozy thing sock sweater fuzzy plushie thing fish pepper

main video game anime DIY boardgame tshirt size Medium believe snack culture thing help interact way wish list 3d printing craft cosplay tool material anime video game figure merch smart home thing snack thing good share Graphic Ts Tee Turtle video game switch steam Board game

travel racquet sport beach coffee

read board game run woodworke pastel neutral shade chocolate especially dark chocolate add plant desk fun

strategy deception board game MCU Deadpool superhero comic_strip coffee tea huge dark chocolate



In [561]:
spacy_corpus = corpus.apply(nlp)
print(spacy_corpus)
spacy_similarity = np.zeros((len(spacy_corpus), len(spacy_corpus)))
for (i, doc) in enumerate(spacy_corpus):
    for (j, other_doc) in enumerate(spacy_corpus):
        spacy_similarity[i, j] = doc.similarity(other_doc)

0     (learn, history, different, country, learn, an...
1     (travel, skydiving, read, red, kitkat, harry, ...
2     (video, game, Pokemon, Animal, Crossing-, plus...
3     (compete, powerlifting, play, electric, guitar...
4     (outside, work, cook, watch, Netflix, Hulu, dr...
5     (electric, engineering, red, any(all, em!!!!11...
6     (passion, tasteful, elegant, knickknack, goodw...
7                            (Tennis, Cricket, Beaches)
8     (eat, travel, running, hiking, cozy, thing, so...
9     (LEGO, bowl, league, 20, year, Star, Wars, wor...
10    (musician, saxophone, red, legos, model, build...
11    (car, game, build, pc, electronic, chocolate, ...
12    (main, video, game, anime, DIY, boardgame, tsh...
13    (Mechanical, Engineering, thing, apart, eatabl...
14    (run, University, Maryland, Sports, green, cof...
15    (bake, cooking, board, game, k, beauty, produc...
16    (winter, wear, wool, sock, hat, big, plus, wil...
17    (run, active, Texas, Tech, read, learn, hi

In [562]:
print(spacy_similarity)

[[1.         0.45793388 0.4867388  ... 0.57037899 0.3732326  0.61234764]
 [0.45793388 1.         0.64056009 ... 0.70777976 0.5286397  0.60534804]
 [0.4867388  0.64056009 1.         ... 0.78303217 0.56858762 0.67523128]
 ...
 [0.57037899 0.70777976 0.78303217 ... 1.         0.63037607 0.71026754]
 [0.3732326  0.5286397  0.56858762 ... 0.63037607 1.         0.55582653]
 [0.61234764 0.60534804 0.67523128 ... 0.71026754 0.55582653 1.        ]]


In [567]:
def check_person(similarity, person):
    idx = 0
    prefs = df["proc"]
    for i, p in enumerate(df["Email Address"]):
        if p == person:
            idx = i

    sim = np.argsort(-similarity[idx,:])
    for i in sim:
        for j, p in enumerate(df["Email Address"]):
            if i == j:
                print(f"{p}: {prefs.loc[i]}")
                
    
check_person(spacy_similarity, "nice.yslas@appian.com")

nice.yslas@appian.com: travel skydiving read red kitkat harry potter caffeine addict
dharti.desh@appian.com: 1 decor enthusiast mean desk join Appian get 2 book read specially fiction thriller etc 3 color(s turquoise orange yellow bright tbh 4 avid follower Marvel DC Harry Potter Game throne
debby.chang@appian.com: travel read collect decorative mug Christmas involve peppermint harry potter kind art learn paint draw
coby.lu@appian.com: strategy deception board game MCU Deadpool superhero comic_strip coffee tea huge dark chocolate
annika.basch@appian.com: outside work cook watch Netflix Hulu draw ipad knit teal green Ferrero Rocher get bath body work product bath bomb gift
annie.zweighaft@appian.com: candle smell forest go ski trip west January foster cat cute holiday cat toy drink ice coffee year round
ryan.probus@appian.com: board game puzzle camping hiking rock climbing motorcycle video game food alcohol blue small gift mind open related desk feel pretty mind desk trinket
rachel.bitt

In [543]:
df_simp

,Email Address,List 4 facts about yourself to help your secret snowflake find the perfect gift for you!,proc
0,sydney.pujadas@appian.com,"foodie, art lover, traveler, music lover",foodie art traveler music
1,rachel.bittner@appian.com,"Hobbies: video games - Pokemon, Animal Crossin...",video game Pokemon Animal Crossing- plushie aw...
2,erin.kocis@appian.com,"I love eating, traveling, running, hiking, coz...",eat travel running hiking cozy thing sock swea...
3,franklin.zheng@appian.com,"My main hobbies are Video games, anime, DIY, b...",main video game anime DIY boardgame tshirt siz...
4,jorge.sanchez@appian.com,"Love to travel, love racquet sports, love beac...",travel racquet sport beach coffee
5,katie.freix@appian.com,"I enjoy reading, board games, running and wood...",read board game run woodworke pastel neutral s...
6,coby.lu@appian.com,I like strategy/deception board games \nI like...,strategy deception board game MCU Deadpool sup...


In [466]:
print(spacy_similarity)

[[1.         0.78004595 0.71379496 0.74694335 0.59000137 0.76675108
  0.8006345 ]
 [0.78004595 1.         0.74574058 0.80378291 0.5019318  0.81442858
  0.82130873]
 [0.71379496 0.74574058 1.         0.74404284 0.66179628 0.75894692
  0.68925618]
 [0.74694335 0.80378291 0.74404284 1.         0.63090381 0.82727095
  0.81316381]
 [0.59000137 0.5019318  0.66179628 0.63090381 1.         0.61291704
  0.58916742]
 [0.76675108 0.81442858 0.75894692 0.82727095 0.61291704 1.
  0.82452935]
 [0.8006345  0.82130873 0.68925618 0.81316381 0.58916742 0.82452935
  1.        ]]


In [299]:
corpus = df["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]

In [ ]:
corpus = df_simp["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]

In [300]:
print(corpus)

0     Enjoy learning about history of different coun...
1     Hobbies: traveling, skydiving, reading.  favor...
2     Hobbies: video games - Pokemon, Animal Crossin...
3     I compete in powerlifting, I play electric gui...
4     Outside of work I like to cook, watch Netflix ...
5     Hobby: electric engineering; color/candy: red/...
6     I have a passion for tasteful and elegant knic...
7                          Tennis, Cricket, Beaches... 
8     I love eating, traveling, running, hiking, coz...
9     I like LEGO, I've been bowling in a league for...
10    I am a musician (saxophone)\nMy favorite color...
11    I love cars and games, I build PCs/electronics...
12    My main hobbies are Video games, anime, DIY, b...
13    Mechanical Engineering\nI like to take things ...
14    I like running and University of Maryland Spor...
15    I love baking/cooking\nI really like board gam...
16    I'm in need of winter wear, so wool socks + ha...
17    I really like to run and keep active, Texa

In [301]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                   ngram_range=(1,2), 
                                   max_df=0.95, 
                                   min_df=2,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(corpus)

In [302]:
foo = list(tfidf_vectorizer.get_stop_words())
foo.sort()
print(foo)

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'do', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give

In [303]:
print(tfidf_vectorizer.get_feature_names())

['art', 'artists', 'baking', 'beauty', 'beauty products', 'big', 'blue', 'board', 'board games', 'books', 'camping', 'candy', 'candy like', 'candy red', 'cars', 'cat', 'chocolate', 'coffee', 'color', 'color blue', 'color candy', 'color green', 'colors', 'cook', 'cooking', 'course', 'craft', 'creative', 'currently', 'dark', 'dark chocolate', 'desk', 'desk trinket', 'dog', 'doing', 'don', 'don like', 'draw', 'eating', 'electric', 'electronics', 'engineering', 'enjoy', 'especially', 'favorite', 'favorite candy', 'favorite color', 'fiction', 'food', 'fun', 'game', 'games', 'games like', 'getting', 'gift', 'gifts', 'good', 'good like', 'green', 'green favorite', 'guitar', 'harry', 'harry potter', 'hiking', 'history', 'hobbies', 'hobbies video', 'home', 'interesting', 'just', 'kids', 'league', 'learning', 'learning history', 'legos', 'like', 'like board', 'like cook', 'like things', 'listening', 'listening music', 'love', 'main', 'mind', 'motorcycles', 'movies', 'mugs', 'music', 'need', 'new

In [304]:
print(tfidf.todense())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.27153622]]


In [305]:
y = squareform(pdist(tfidf.todense(), "cosine"))

In [306]:
print(y)

[[0.         1.         1.         ... 1.         1.         1.        ]
 [1.         0.         0.86935747 ... 0.96094394 0.94791019 0.81381523]
 [1.         0.86935747 0.         ... 0.91887773 1.         0.93275491]
 ...
 [1.         0.96094394 0.91887773 ... 0.         1.         0.82624647]
 [1.         0.94791019 1.         ... 1.         0.         0.9572854 ]
 [1.         0.81381523 0.93275491 ... 0.82624647 0.9572854  0.        ]]


In [295]:
def check_person(similarity, person):
    idx = 0
    prefs = df_simp["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]
    for i, p in enumerate(df_simp["Email Address"]):
        if p == person:
            idx = i

    sim = np.argsort(similarity[idx,:])
    for i in sim:
        for j, p in enumerate(df_simp["Email Address"]):
            if i == j:
                print(f"{p}: {prefs.loc[i]}")
                
    
check_person(y, "rachel.bittner@appian.com")

rachel.bittner@appian.com: Hobbies: video games - Pokemon, Animal Crossing- plushies are awesome
Candy/Color: Sour Candy/Blue- sour patch kids/sour punch straws
franklin.zheng@appian.com: My main hobbies are Video games, anime, DIY, boardgames.
My Tshirt size is Medium
I believe in snack culture
I enjoy things that help me interact with others in an interesting way

Wish list:
3d printing/craft/cosplay tools/materials
anime/video game figures and merch
smart home things
Snacks/candy! (Things that are good to share)
Graphic Ts (like from Tee Turtle or anything)
Video games (like switch or on steam)
Board games
nice.yslas@appian.com: Hobbies: traveling, skydiving, reading.  favorite color/candy: red/kitkat, i love harry potter. caffeine addict. 
coby.lu@appian.com: I like strategy/deception board games 
I like MCU, Deadpool, various superhero comics
I like coffee/tea
I'm not huge on candy, but I like dark chocolate
katie.freix@appian.com: I enjoy reading, board games, running and woodwor

In [314]:
print(y.shape[0])

42


In [311]:
num_people = len(corpus)
c = np.reshape(y, -1)

for (i, doc) in enumerate(corpus):
    for (j, other_doc) in enumerate(corpus):
        c[i * num_people + j] = c[i * num_people + j] if i != j else 99999

In [320]:
for i in c:
    print(i)

99999.0
1.0
1.0
1.0
1.0
1.0
nan
nan
1.0
1.0
0.8481790408935925
1.0
0.9262735863453134
1.0
1.0
1.0
1.0
0.5996787874747392
1.0
1.0
1.0
0.7184531360921569
0.7790817483256798
0.8009979029006749
0.8928904914735292
1.0
1.0
1.0
1.0
1.0
0.8104764015547129
0.8875388321375561
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
99999.0
0.8693574710119799
0.9565790178800402
0.8217485809439191
0.6071653447779428
nan
nan
0.904683639163862
1.0
0.7604978072962911
0.9656729042196868
0.9373349914168484
1.0
0.8347867861146546
0.9727672770061541
1.0
0.9379711873817071
1.0
1.0
0.8928803685653499
1.0
0.8543167413564579
1.0
0.970712693758854
0.7916354267811669
0.8410966313048523
0.9661590509322899
0.8859663169993931
1.0
0.45669755000693046
0.8462427805630659
0.9701580035358833
0.5223466169555213
1.0
1.0
0.7569052137309072
0.9540513885383122
1.0
0.960943940065724
0.9479101899182273
0.81381522914425
1.0
0.8693574710119799
99999.0
1.0
0.9356202307382296
0.9177299168977444
nan
nan
0.9357687149676451
1.0
0.97198576751537

0.6964339711028955
1.0
1.0
0.7577723953210005
1.0
0.9302728214460271
0.9211119144102617
99999.0
1.0
1.0
0.85901535861871
0.798832947971639
0.8692372264886911
1.0
1.0
0.7569052137309072
0.9591653028688862
0.9286240450259023
0.7365532924328833
0.9392930111404343
nan
nan
0.9036887053111198
0.9297821985511079
0.7788908634976339
0.9435726894422112
0.964992669905137
0.888320237318917
0.7342435980018307
0.9248097690260672
1.0
0.9693979239991032
0.9658764295774012
0.8941737009899988
0.40794410754611876
0.9665784502683157
0.9038663436433644
0.9403363812730847
0.9518571586933378
0.8725984834567904
0.6804102223723254
0.9065642911524775
0.8641077792147103
1.0
0.857818903935481
0.7828140521321733
0.9176054995540737
0.7639813898727886
0.9173516616633014
1.0
99999.0
0.8731347317153505
1.0
0.882104941289733
0.9143741169019568
0.724828194127007
1.0
0.9540513885383122
1.0
0.6629732190215605
0.9394097308849152
1.0
nan
nan
0.8155763841188066
0.931393195615535
1.0
0.944867293229091
0.9657958381167949
0.769

In [310]:
print(c.shape)
print(num_people**2)

(1764,)
1764


In [ ]:
df_simp = df.loc[[1, 2, 8, 12, 20, 31, 39]].reset_index(drop=True)
df_simp.head(7)

In [ ]:
corpus = df_simp["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]

In [ ]:
spacy_corpus = corpus.apply(nlp)

In [401]:
num_people = len(corpus)
c = np.reshape(spacy_similarity, -1)


In [402]:
# give to only 1
give = np.zeros((num_people, num_people**2))
for i in range(num_people):
    give[i, i * num_people:(i + 1) * num_people] = 1
print(give)

[[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
  1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  

In [403]:
# receive from only 1
receive = np.zeros((num_people, num_people**2))
for i in range(num_people):
    receive[i, i:num_people ** 2:num_people] = 1
print(receive)

[[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
  0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  

In [404]:
# don't give and receive from same person
giv_rec = np.zeros((num_people * (num_people - 1), num_people**2))
from itertools import combinations
i = 0
for comb in combinations(range(num_people), 2):
    giv_rec[i, comb[0] * num_people + comb[1]] = 1
    giv_rec[i, comb[1] * num_people + comb[0]] = 1
    i += 1
print(giv_rec[0])

[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


In [405]:
A_eq = np.vstack((give, receive))

In [406]:
print(A_eq.shape)

(14, 49)


In [407]:
b_eq = np.ones(2 * num_people)
print(b_eq.shape)

(14,)


In [408]:
A_ub = giv_rec
b_ub = np.ones((num_people - 1) * num_people)
print(A_ub.shape)
print(b_ub.shape)

(42, 49)
(42,)


In [409]:
print(c.shape)

(49,)


In [410]:
from scipy.optimize import linprog

In [411]:
res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, method="revised simplex")

/Users/coby.lu/repo/secret-snowflake/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """Entry point for launching an IPython kernel.


In [412]:
print(res)

     con: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
     fun: 4.778108637235652
 message: 'Optimization terminated successfully.'
     nit: 42
   slack: array([1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])
  status: 0
 success: True
       x: array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])


In [413]:
solution = np.round(res.x)
members = df_simp["Email Address"]
for i, g in enumerate(members):
    for j, r in enumerate(members):
        if solution[i * num_people + j] == 1:
            print(f"{g} gives to {r}")

nice.yslas@appian.com gives to katie.freix@appian.com
rachel.bittner@appian.com gives to jorge.sanchez@appian.com
erin.kocis@appian.com gives to franklin.zheng@appian.com
franklin.zheng@appian.com gives to nice.yslas@appian.com
jorge.sanchez@appian.com gives to coby.lu@appian.com
katie.freix@appian.com gives to rachel.bittner@appian.com
coby.lu@appian.com gives to erin.kocis@appian.com


In [416]:
print(spacy_similarity)

[[1.         0.77749909 0.68901754 0.72977826 0.56326649 0.77256937
  0.79377719]
 [0.77749909 1.         0.71606993 0.8092341  0.51435234 0.81424469
  0.80941627]
 [0.68901754 0.71606993 1.         0.72301873 0.62651734 0.76755394
  0.63821163]
 [0.72977826 0.8092341  0.72301873 1.         0.6426177  0.83222097
  0.8164446 ]
 [0.56326649 0.51435234 0.62651734 0.6426177  1.         0.62862653
  0.58593362]
 [0.77256937 0.81424469 0.76755394 0.83222097 0.62862653 1.
  0.80815206]
 [0.79377719 0.80941627 0.63821163 0.8164446  0.58593362 0.80815206
  1.        ]]


In [415]:
def check_person(spacy_similarity, person):
    idx = 0
    prefs = df_simp["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]
    for i, p in enumerate(df_simp["Email Address"]):
        if p == person:
            idx = i

    sim = np.argsort(-spacy_similarity[idx,:])
    for i in sim:
        for j, p in enumerate(df_simp["Email Address"]):
            if i == j:
                print(f"{p}: {prefs.loc[i]}")
                
    
check_person(spacy_similarity, "nice.yslas@appian.com")

nice.yslas@appian.com: Hobbies: traveling, skydiving, reading.  favorite color/candy: red/kitkat, i love harry potter. caffeine addict. 
coby.lu@appian.com: I like strategy/deception board games 
I like MCU, Deadpool, various superhero comics
I like coffee/tea
I'm not huge on candy, but I like dark chocolate
rachel.bittner@appian.com: Hobbies: video games - Pokemon, Animal Crossing- plushies are awesome
Candy/Color: Sour Candy/Blue- sour patch kids/sour punch straws
katie.freix@appian.com: I enjoy reading, board games, running and woodworking 
I don't really have a favorite color but I like any pastel/neutral shade
I like chocolate, especially dark chocolate :)
Adding plants to my desk could be fun!
franklin.zheng@appian.com: My main hobbies are Video games, anime, DIY, boardgames.
My Tshirt size is Medium
I believe in snack culture
I enjoy things that help me interact with others in an interesting way

Wish list:
3d printing/craft/cosplay tools/materials
anime/video game figures and m

In [546]:
tfidf.todense()

(42, 139)

In [551]:
np.hstack((tfidf.todense(), np.ones((tfidf.shape[0], 1))))

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.27153622,
         1.        ]])

In [568]:
num_people = 3
# give to only 1
give = np.zeros((num_people, num_people**2))
for i in range(num_people):
    give[i, i * num_people:(i + 1) * num_people] = 1

# receive from only 1
receive = np.zeros((num_people, num_people**2))
for i in range(num_people):
    receive[i, i:num_people ** 2:num_people] = 1

In [569]:
print(give)

[[1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1.]]


In [570]:
print(receive)

[[1. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1.]]


In [571]:
giv_rec = np.zeros((num_people * (num_people - 1), num_people**2))
for i, comb in enumerate(combinations(range(num_people), 2)):
    giv_rec[i, comb[0] * num_people + comb[1]] = 1
    giv_rec[i, comb[1] * num_people + comb[0]] = 1
print(giv_rec)

[[0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
#TFIDF
kunal.chitalia@appian.com gives to kevin.reed@appian.com
nice.yslas@appian.com gives to dharti.desh@appian.com
rachel.bittner@appian.com gives to katie.ward@appian.com
annika.basch@appian.com gives to abigail.butler@appian.com
dima.chechetkin@appian.com gives to brandon.long@appian.com
arul.josely@appian.com gives to tom.zanger@appian.com
erin.kocis@appian.com gives to kevin.lin@appian.com
aaron.greb@appian.com gives to bill.warshaw@appian.com
brandon.long@appian.com gives to kelley.macewen@appian.com
drew.kidwell@appian.com gives to michael.nye@appian.com
franklin.zheng@appian.com gives to brett.shomaker@appian.com
patrick.hanley@appian.com gives to christine.danzi@appian.com
cynthia.wu@appian.com gives to katie.freix@appian.com
kevin.reed@appian.com gives to aaron.greb@appian.com
jorge.sanchez@appian.com gives to annie.zweighaft@appian.com
serdar.benderli@appian.com gives to srikanth.chelluri@appian.com
bill.warshaw@appian.com gives to kunal.chitalia@appian.com
kelley.macewen@appian.com gives to patrick.hanley@appian.com
abigail.butler@appian.com gives to rachel.bittner@appian.com
scheherbano.rafay@appian.com gives to annika.basch@appian.com
ryan.probus@appian.com gives to michael.cassic@appian.com
debby.chang@appian.com gives to nice.yslas@appian.com
katie.freix@appian.com gives to coby.lu@appian.com
lyndsey.ferguson@appian.com gives to derika.chia@appian.com
dharti.desh@appian.com gives to debby.chang@appian.com
brett.shomaker@appian.com gives to drew.kidwell@appian.com
michael.cassic@appian.com gives to franklin.zheng@appian.com
isabel.trumbull@appian.com gives to jorge.sanchez@appian.com
annie.zweighaft@appian.com gives to jake.shankman@appian.com
pedram.fard@appian.com gives to dima.chechetkin@appian.com
coby.lu@appian.com gives to cynthia.wu@appian.com
derika.chia@appian.com gives to scheherbano.rafay@appian.com
christine.danzi@appian.com gives to isabel.trumbull@appian.com

In [1]:
from itertools import combinations

import numpy as np
import pandas as pd
from scipy.optimize import linprog
import spacy

In [56]:
def similarity(df):
    nlp = spacy.load("en_core_web_md")
    stop_lemmas = set(['\n', '\n\n', ' ', '!', '"', '&', "'", '(', ')', '+', ',', 
    '-', '--', '-PRON-', '.', '..', '...', '/', ':', ':)', ';', '>', '?','_', 'a', 
    'an', 'also', 'candy', 'color', 'enjoy', 'favorite', 'hobby', 'honestly', 'i',
    'interested', 'interesting', 'into', 'like', 'love', 'lover', 'need', 
    'preferably','obsessed','really'])

    corpus = df["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]
    spacy_corpus = corpus.apply(nlp)
    for i, sentence in enumerate(spacy_corpus):
        new_sentence = []
        for word in sentence:
            if not word.is_stop and str(word.lemma_) not in stop_lemmas:
                new_sentence.append(str(word.lemma_))
        df.loc[i, "interests (cleaned)"] = " ".join(new_sentence)

    corpus = df["interests (cleaned)"]
    spacy_corpus = corpus.apply(nlp)
    spacy_similarity = np.zeros((len(spacy_corpus), len(spacy_corpus)))
    for (i, doc) in enumerate(spacy_corpus):
        for (j, other_doc) in enumerate(spacy_corpus):
            spacy_similarity[i, j] = doc.similarity(other_doc)
    
    return -spacy_similarity

def optimize(similarity):
    num_people = similarity.shape[0]
    print(f"num people1: {num_people}")

    c = np.reshape(similarity, -1)
    for i in range(num_people):
        for j in range(num_people):
            c[i * num_people + j] = c[i * num_people + j] if i != j else 99999
    print(c)
    print(c.shape)
    
    # give to only 1
    give = np.zeros((num_people, num_people**2))
    for i in range(num_people):
        give[i, i * num_people:(i + 1) * num_people] = 1
    print(all(sum(give) == np.ones(1764)))
    
    # receive from only 1
    receive = np.zeros((num_people, num_people**2))
    for i in range(num_people):
        receive[i, i:num_people ** 2:num_people] = 1
    print(all(sum(receive) == np.ones(1764)))

    # don't give and receive from same person
    giv_rec = np.zeros((num_people * (num_people - 1) // 2, num_people**2))
    for i, comb in enumerate(combinations(range(num_people), 2)):
        giv_rec[i, comb[0] * num_people + comb[1]] = 1
        giv_rec[i, comb[1] * num_people + comb[0]] = 1
    print(sum(sum(giv_rec)))

    A_eq = np.vstack((give, receive))
    print(A_eq.shape)
    b_eq = np.ones(2 * num_people)

    A_ub = giv_rec
    print(A_ub.shape)
    b_ub = np.ones((num_people - 1) * num_people // 2)
#     return None
    return linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=(0, 1))

In [5]:
df = pd.read_csv("Engineering Secret Snowflake 2019 (Responses) - Form Responses 1.csv")
going = df.loc[df["Do you want to participate in Engineering's Secret Snowflake?"] == "Yes", ["Email Address", "List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]].reset_index(drop=True)

interest_similarity = similarity(going)

In [57]:
result = optimize(interest_similarity)

num people1: 42
[ 9.99990000e+04 -4.57933877e-01 -4.86738897e-01 ... -7.10267518e-01
 -5.55826535e-01  9.99990000e+04]
(1764,)
True
True
1722.0
(84, 1764)
(861, 1764)


/Users/coby.lu/repo/secret-snowflake/venv/lib/python3.7/site-packages/ipykernel_launcher.py:65: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


In [32]:
result

     con: array([-4.44089210e-16, -2.22044605e-16,  3.33066907e-16, -4.44089210e-16,
        1.33226763e-15,  5.55111512e-16, -2.22044605e-16, -4.44089210e-16,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -4.44089210e-16,
       -2.22044605e-16,  2.22044605e-16, -4.44089210e-16, -8.88178420e-16,
        0.00000000e+00,  2.22044605e-16, -2.22044605e-16, -8.88178420e-16,
       -2.22044605e-16,  0.00000000e+00,  1.11022302e-16,  2.22044605e-16,
       -8.88178420e-16, -2.22044605e-16,  0.00000000e+00,  1.11022302e-16,
        3.33066907e-16, -4.44089210e-16,  0.00000000e+00,  8.88178420e-16,
        4.44089210e-16, -4.44089210e-16, -4.44089210e-16,  8.88178420e-16,
       -2.22044605e-16,  2.22044605e-16, -4.44089210e-16,  3.33066907e-16,
        4.44089210e-16,  7.77156117e-16, -6.66133815e-16, -2.22044605e-16,
        5.55111512e-16, -2.22044605e-16,  2.22044605e-16,  4.44089210e-16,
       -2.22044605e-16, -2.22044605e-16,  6.66133815e-16,  0.00000000e+00,
       -1.11022

In [34]:
solution = np.round(result.x)

In [41]:
print(sum(result.x))
print(sum(solution))

41.99999999999999
40.0


In [36]:
result.x

array([0., 0., 0., ..., 0., 0., 0.])

In [58]:
count = 0
members = going["Email Address"]
num_people = len(members)
for i, g in enumerate(members):
    for j, r in enumerate(members):
        if 0.9 > result.x[i * num_people + j] > 0.1:
            print(f"{i} gives to {j}: {result.x[i * num_people + j]}")
            count += 1
print(count)

0 gives to 17: 0.5000002095043663
0 gives to 36: 0.4999997904061633
1 gives to 30: 0.49999992736678894
1 gives to 33: 0.5000000725444849
2 gives to 23: 0.5000000660823477
2 gives to 39: 0.4999999338340598
3 gives to 10: 0.5000000658425807
3 gives to 18: 0.49999993385549296
4 gives to 40: 0.4999999958196277
4 gives to 41: 0.5000000037334635
5 gives to 13: 0.4999999956230986
5 gives to 40: 0.5000000040983082
6 gives to 7: 0.49999979043335624
6 gives to 36: 0.5000002091792324
7 gives to 6: 0.5000002094566534
7 gives to 20: 0.49999979048778287
8 gives to 16: 0.4999998565983832
8 gives to 37: 0.5000001433167431
9 gives to 23: 0.49999993382748287
9 gives to 38: 0.5000000657282336
10 gives to 3: 0.49999993358967415
10 gives to 11: 0.5000000661210532
11 gives to 10: 0.499999933868146
11 gives to 24: 0.5000000657883759
12 gives to 15: 0.4999999991546006
12 gives to 19: 0.5000000004536881
13 gives to 5: 0.5000000039612592
13 gives to 27: 0.4999999958018556
14 gives to 20: 0.5000002094637119
14 g

In [46]:
print(interest_similarity[8, [16, 37]])

[-0.84697069 -0.83017431]


In [48]:
print(interest_similarity[16, [8, 25]])

[-0.84697069 -0.78766919]


In [49]:
print(interest_similarity[25, [16, 31]])

[-0.78766919 -0.80605586]


In [50]:
print(interest_similarity[26, [13, 37]])

[-0.83021198 -0.8668073 ]


In [51]:
print(interest_similarity[31, [39, 25]])

[-0.82353824 -0.80605586]


In [52]:
print(interest_similarity[35, [31, 39]])

[-0.88030439 -0.83383503]


In [53]:
print(interest_similarity[37, [8, 14]])

[-0.83017431 -0.77130344]


In [55]:
result.slack.shape

(861,)

In [2]:
from itertools import combinations

import numpy as np
import pandas as pd
from pulp import *
from scipy.optimize import linprog
import spacy

In [3]:
def similarity(df):
    nlp = spacy.load("en_core_web_md")
    stop_lemmas = set(['\n', '\n\n', ' ', '!', '"', '&', "'", '(', ')', '+', ',', 
    '-', '--', '-PRON-', '.', '..', '...', '/', ':', ':)', ';', '>', '?','_', 'a', 
    'an', 'also', 'candy', 'color', 'enjoy', 'favorite', 'hobby', 'honestly', 'i',
    'interested', 'interesting', 'into', 'like', 'love', 'lover', 'need', 
    'preferably','obsessed','really'])

    corpus = df["List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]
    spacy_corpus = corpus.apply(nlp)
    for i, sentence in enumerate(spacy_corpus):
        new_sentence = []
        for word in sentence:
            if not word.is_stop and str(word.lemma_) not in stop_lemmas:
                new_sentence.append(str(word.lemma_))
        df.loc[i, "interests (cleaned)"] = " ".join(new_sentence)

    corpus = df["interests (cleaned)"]
    spacy_corpus = corpus.apply(nlp)
    spacy_similarity = np.zeros((len(spacy_corpus), len(spacy_corpus)))
    for (i, doc) in enumerate(spacy_corpus):
        for (j, other_doc) in enumerate(spacy_corpus):
            spacy_similarity[i, j] = doc.similarity(other_doc)
    
    return -spacy_similarity

In [4]:
df = pd.read_csv("Engineering Secret Snowflake 2019 (Responses) - Form Responses 1.csv")
going = df.loc[df["Do you want to participate in Engineering's Secret Snowflake?"] == "Yes", ["Email Address", "List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]].reset_index(drop=True)

interest_similarity = similarity(going)

In [6]:
num_people = interest_similarity.shape[0]

In [13]:
def optimize(similarity):
    num_people = similarity.shape[0]
    print(f"num people1: {num_people}")

    lp_vars = []
    c = []
    for i in range(num_people):
        for j in range(num_people):
            lp_vars.append(LpVariable(f"g_{i}_r_{j}", lowBound = 0, upBound = 1, cat=LpInteger))
            c.append(similarity[i][j] if i != j else 99999)
    
    prob = LpProblem("snowflake", LpMinimize)
    
    # minimization function
    prob += lpDot(c, lp_vars)

    # give only 1
    for i in range(num_people):
        prob += lpSum([lp_vars[i] for i in range(i * num_people, (i + 1) * num_people)]) == 1

    # receive only 1
    for i in range(num_people):
        prob += lpSum([lp_vars[i] for i in range(i, num_people ** 2, num_people)]) == 1

    # don't give and receive to the same person
    for comb in combinations(range(num_people), 2):
        prob += lp_vars[comb[0] * num_people + comb[1]] + lp_vars[comb[1] * num_people + comb[0]] <= 1
        
    return lp_vars, prob.solve()

In [17]:
lp_vars, result = optimize(interest_similarity)

num people1: 42


In [21]:
count = 0
for v in lp_vars:
    if v.varValue > 0.9:
        print(v.__dict__)
        count += 1
        
print(count)

{'_LpElement__name': 'g_0_r_17', 'hash': 5084085968, 'modified': True, 'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 1.0, 'dj': -0.77052935, 'init': 0}
{'_LpElement__name': 'g_1_r_30', 'hash': 5094771280, 'modified': True, 'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 1.0, 'dj': -0.75230492, 'init': 0}
{'_LpElement__name': 'g_2_r_23', 'hash': 5094956048, 'modified': True, 'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 1.0, 'dj': -0.73626886, 'init': 0}
{'_LpElement__name': 'g_3_r_10', 'hash': 5094755024, 'modified': True, 'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 1.0, 'dj': -0.75869693, 'init': 0}
{'_LpElement__name': 'g_4_r_40', 'hash': 5094760272, 'modified': True, 'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 1.0, 'dj': -0.68513992, 'init': 0}
{'_LpElement__name': 'g_5_r_13', 'hash': 5094831888, 'modified': True, 'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 1.0, 'dj': -0.73298947, 'init': 0}
{'_LpEleme

In [22]:
print(LpStatus[result])

Optimal


In [26]:
going.loc[2, "List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]

'Hobbies: video games - Pokemon, Animal Crossing- plushies are awesome\r\nCandy/Color: Sour Candy/Blue- sour patch kids/sour punch straws'

In [30]:
import csv
print(LpStatus[result])
members = going["Email Address"]
num_people = len(members)
check = 0
matching = pd.DataFrame(columns = ["Giver", "Receiver", "Giver's interests", "Receiver's Interests"])
for i, g in enumerate(members):
    for j, r in enumerate(members):
        if lp_vars[i * num_people + j].varValue  >= 0.95:
            matching = matching.append({"Giver": g, "Receiver": r, "Giver's interests": going.loc[i, "List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "], "Receiver's Interests": going.loc[j, "List 4 facts about yourself to help your secret snowflake find the perfect gift for you! "]}, ignore_index = True)
            check += 1
matching.to_csv("matching.csv",index = False, quoting=csv.QUOTE_ALL)
print(f"{check} number of snowflakes matched")

Optimal
42 number of snowflakes matched
